# Data Pre-processing

Rewriteing the products into a standard format.  


In [2]:
# imports
from litellm import completion
from dotenv import load_dotenv
import json
from pricer.batch import Batch
from pricer.items import Item

load_dotenv(override=True)

True

In [3]:

dataset = f"ed-donner/items_raw_lite"

train, val, test = Item.from_hub(dataset)

items = train + val + test

print(f"Loaded {len(items):,} items")
print(items[0])

Loaded 22,000 items
title='Schlage F59 AND 613 Andover Interior Knob with Deadbolt, Oil Rubbed Bronze (Interior Half Only)' category='Tools_and_Home_Improvement' price=64.3 full='Schlage F59 AND 613 Andover Interior Knob with Deadbolt, Oil Rubbed Bronze (Interior Half Only)\n[\'From the Manufacturer\', "When you have a Schlage handleset on your front door, you ensure your security as well as your peace of mind. After all, we\'re the leader in security devices, trusted for over 85 years. All Schlage handlesets are precision engineered, featuring 100% solid"]\n[\'Interior half only\', \'Requires F58 to complete handle set\', \'Non handed knob style\', \'4" minimum center to center door prep required for this two piece model.\', \'Lifetime Mechanical and Finish Warranty\']\n{"Material": "Metal", "Brand": "", "Color": "Oil Rubbed Bronze", "Exterior Finish": "Bronze", "Special Feature": "Easy to Install", "Age Range (Description)": "Adult", "Included Components": "Deadbolt, Knob", "Item Wei

In [5]:
# Give every item an id

for index, item in enumerate(items):
    item.id = index

In [6]:
SYSTEM_PROMPT = """Create a concise description of a product. Respond only in this format. Do not include part numbers.
Title: Rewritten short precise title
Category: eg Electronics
Brand: Brand name
Description: 1 sentence description
Details: 1 sentence on features"""

In [7]:
print(items[0].full)

Schlage F59 AND 613 Andover Interior Knob with Deadbolt, Oil Rubbed Bronze (Interior Half Only)
['From the Manufacturer', "When you have a Schlage handleset on your front door, you ensure your security as well as your peace of mind. After all, we're the leader in security devices, trusted for over 85 years. All Schlage handlesets are precision engineered, featuring 100% solid"]
['Interior half only', 'Requires F58 to complete handle set', 'Non handed knob style', '4" minimum center to center door prep required for this two piece model.', 'Lifetime Mechanical and Finish Warranty']
{"Material": "Metal", "Brand": "", "Color": "Oil Rubbed Bronze", "Exterior Finish": "Bronze", "Special Feature": "Easy to Install", "Age Range (Description)": "Adult", "Included Components": "Deadbolt, Knob", "Item Weight": "1.5 pounds", "Handle Material": "Bronze", "Package Type": "Standard Packaging", "Unit Count": "1.0 Count", "Number of Items": "1", "Manufacturer": "Schlage", "Product Dimensions": "8.1 x 4

In [8]:
messages = [{"role": "system", "content": SYSTEM_PROMPT}, {"role": "user", "content": items[0].full}]
response = completion(messages=messages, model="groq/openai/gpt-oss-20b", reasoning_effort="low")

print(response.choices[0].message.content)
print()
print(f"Input tokens: {response.usage.prompt_tokens}")
print(f"Output tokens: {response.usage.completion_tokens}")
print(f"Cost: {response._hidden_params['response_cost']*100:.3f} cents")

Title: Schlage F59 Interior Knob with Deadbolt, Oil Rubbed Bronze  
Category: Hardware  
Brand: Schlage  
Description: A durable interior half‑door handle set featuring an oil‑rubbed bronze knob and integrated deadbolt for enhanced security.  
Details: Designed for a 4″ center‑to‑center door prep, this metal knob includes a deadbolt, is easy to install, and comes with a lifetime mechanical and finish warranty.

Input tokens: 446
Output tokens: 153
Cost: 0.008 cents


In [9]:
MODEL = "openai/gpt-oss-20b"

In [10]:
def make_jsonl(item):
    body = {"model": MODEL, "messages": [{"role": "system", "content": SYSTEM_PROMPT}, {"role": "user", "content": item.full}], "reasoning_effort": "low"}
    line = {"custom_id": str(item.id), "method": "POST", "url": "/v1/chat/completions", "body": body}
    return json.dumps(line)

In [11]:
items[0]

<Schlage F59 AND 613 Andover Interior Knob with Deadbolt, Oil Rubbed Bronze (Interior Half Only) = $64.3>

In [12]:
make_jsonl(items[0])

'{"custom_id": "0", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "openai/gpt-oss-20b", "messages": [{"role": "system", "content": "Create a concise description of a product. Respond only in this format. Do not include part numbers.\\nTitle: Rewritten short precise title\\nCategory: eg Electronics\\nBrand: Brand name\\nDescription: 1 sentence description\\nDetails: 1 sentence on features"}, {"role": "user", "content": "Schlage F59 AND 613 Andover Interior Knob with Deadbolt, Oil Rubbed Bronze (Interior Half Only)\\n[\'From the Manufacturer\', \\"When you have a Schlage handleset on your front door, you ensure your security as well as your peace of mind. After all, we\'re the leader in security devices, trusted for over 85 years. All Schlage handlesets are precision engineered, featuring 100% solid\\"]\\n[\'Interior half only\', \'Requires F58 to complete handle set\', \'Non handed knob style\', \'4\\" minimum center to center door prep required for this two piece m

In [15]:

def make_file(start, end, filename):
    batch_file = filename
    with open(batch_file, "w") as f:
        for i in range(start, end):
            f.write(make_jsonl(items[i]))
            f.write("\n")

In [16]:
make_file(0, 1000, "jsonl/0_1000.jsonl")

In [17]:
import os
from groq import Groq

groq = Groq(api_key=os.environ.get("GROQ_API_KEY"))

In [ ]:

with open("jsonl/0_1000.jsonl", "rb") as f:
    response = groq.files.create(file=f, purpose="batch")
response

In [ ]:
file_id = response.id
file_id

In [ ]:
response = groq.batches.create(completion_window="24h", endpoint="/v1/chat/completions", input_file_id=file_id)
response

In [ ]:
result = groq.batches.retrieve(response.id)
result

In [ ]:
response = groq.files.content(result.output_file_id)
response.write_to_file("jsonl/batch_results.jsonl")

In [ ]:
with open("jsonl/batch_results.jsonl", "r") as f:
    for line in f:
        json_line = json.loads(line)
        id = int(json_line["custom_id"])
        summary = json_line["response"]["body"]["choices"][0]["message"]["content"]
        items[id].summary = summary


In [ ]:
print(items[0].full)

In [ ]:
print(items[1000].summary)

## I've put exactly this logic into a Batch class

- Divides items into groups of 1,000
- Kicks off batches for each
- Allows us to monitor and collect the results when complete

In [ ]:
Batch.create(items, True)

In [ ]:
Batch.run()

In [ ]:
Batch.fetch()

In [ ]:
# Remove the fields that we don't need in the hub

for item in items:
    item.full = None
    item.id = None